In [1]:
#export
from .imports import *
from .core import *
from .graph import *

In [231]:
np.set_printoptions(threshold=sys.maxsize)

In [196]:
#export
def complete_dag(n:int):
    """
    Generate a complete directed acyclic graph, which corresponds to architecture of ResNet.
    """
    G = nx.DiGraph()
    nodes = list(range(n))
    for id in nodes:
        for succ in range(id+1, n):
            G.add_edge(id, succ)    
    return G

In [472]:
#export
def dual_dag(n:int, d:int=2):
    """
    Generate a DAG, which corresponds to architecture of ResNetX when t=2.
    """
    G = nx.DiGraph()
    nodes = list(range(n))
    for id in nodes[:-1]: 
        G.add_edge(id, id+1) # the backbone chain
        G.add_edge(0, id+1)  # the input node (0) link to all other nodes
        dest = id + 1 + d
        while dest < n:
            G.add_edge(id, dest)
            dest += d

    return G

In [473]:
#export
def triple_dag(n:int):
    """
    Generate a DAG, which corresponds to architecture of ResNetX when t=3.
    """
    G = nx.DiGraph()
    nodes = list(range(n))
    for id in nodes[:-1]:  
        G.add_edge(id, id+1)  # the backbone chain
        G.add_edge(0, id+1)  # the input node (0) link to all other nodes
        if id % 4 == 0 or id % 4 == 2:
            dest = id + 1 + 4
            while dest < n:
                G.add_edge(id, dest)
                dest += 4
        elif id % 4 == 1 or id % 4 == 3:
            dest = id + 1 + 2
            while dest < n:
                G.add_edge(id, dest)
                dest += 2

    return G

In [497]:
#export
def quad_dag(n:int):
    """
    Generate a quad DAG.
    """
    G = nx.DiGraph()
    nodes = list(range(n))
    for id in nodes[:-1]:  
        G.add_edge(id, id+1)  # the backbone chain
        G.add_edge(0, id+1)  # the input node (0) link to all other nodes
        if id % 6 == 0 or id % 6 == 3:
            dest = id + 1 + 6
            while dest < n:
                G.add_edge(id, dest)
                dest += 6
        elif id % 6 == 1 or id % 6 == 5:
            odd_even = 0
            dest = id + 1 + 2
            while dest < n:
                G.add_edge(id, dest)
                odd_even += 1
                if odd_even % 2 == 1:
                    dest += 4
                elif odd_even % 2 == 0:
                    dest += 2
        elif id % 6 == 2 or id % 6 == 4:
            odd_even = 0
            dest = id + 1 + 4
            while dest < n:
                G.add_edge(id, dest)
                odd_even += 1
                if odd_even % 2 == 1:
                    dest += 2
                elif odd_even % 2 == 0:
                    dest += 4

    return G

In [498]:
#export
def incoherent(G):
    mat = nx.to_numpy_matrix(G, nodelist=sorted(G.nodes()))

    in_degree = np.array([item[1] for item in sorted(G.in_degree())])
    in_degree[0] = 1
    Gamma = np.diag(in_degree) - mat
    levels = in_degree @ np.linalg.inv(Gamma)

    levels = np.array(levels).squeeze()

    temp = np.multiply(np.subtract.outer(levels, levels.T).T, mat)

    temp2 = temp[temp != 0]

    avg = np.mean(temp2)

    std1 = np.sqrt(np.mean(np.array(temp2).squeeze() ** 2) - 1)

    std2 = np.std(temp2)
    return avg, std1, std2

In [499]:
n = 20
incoherent(quad_dag(n)), incoherent(triple_dag(n)), incoherent(dual_dag(n)), incoherent(complete_dag(n)), incoherent(dual_dag(n,4))

((1.0, 0.9124149792463323, 0.9124149792463325),
 (1.0, 0.8950274892513621, 0.8950274892513621),
 (0.9999999999999999, 0.8904586637336326, 0.8904586637336329),
 (1.0, 0.8523812059757445, 0.8523812059757446),
 (1.0, 0.925688791771369, 0.9256887917713688))

In [492]:
incoherent(dual_dag(n, d=18))

(1.0, 0.9396186967500834, 0.9396186967500834)

In [495]:
G = dual_dag(20, d=4)
dual_paths = paths_DAG(G)
dual_paths

OrderedDict([(1, 1),
             (2, 5),
             (3, 15),
             (4, 20),
             (5, 35),
             (6, 56),
             (7, 84),
             (8, 36),
             (9, 45),
             (10, 55),
             (11, 66),
             (12, 12),
             (13, 13),
             (14, 14),
             (15, 15),
             (16, 1),
             (17, 1),
             (18, 1),
             (19, 1)])

In [496]:
G = quad_dag(20)
dual_paths = paths_DAG(G)
dual_paths

OrderedDict([(1, 1),
             (2, 6),
             (3, 15),
             (4, 22),
             (5, 51),
             (6, 57),
             (7, 94),
             (8, 64),
             (9, 97),
             (10, 55),
             (11, 80),
             (12, 35),
             (13, 45),
             (14, 14),
             (15, 18),
             (16, 5),
             (17, 6),
             (18, 1),
             (19, 1)])

### Export

In [503]:
!python notebook2script.py graphstats.ipynb

Converted graphstats.ipynb to zero/graphstats.py


### Misc

In [158]:
def hole_dag(n:int, d:int):
    """
    Generate a hole DAG.
    """
    G = nx.DiGraph()
    nodes = list(range(n))
    for id in nodes[:-1]:  # add the chain
        G.add_edge(id, id+1)
        #G.add_edge(0, id+1)
        G.add_edge(id, n-1)
    for i in range(d):
        for j in range(i+1, n):
            G.add_edge(i,j)
            #G.add_edge(n-1-j,n-1-i)

    return G

In [446]:
G = dual_dag(20,3)
mat = nx.to_numpy_matrix(G, nodelist=sorted(G.nodes()))

In [415]:
paths_DAG(G).values(), sum(paths_DAG(G).values())

(odict_values([0, 0, 6]), 6)

In [214]:
levels = [1] #initialize
for i in range(1,9):
    ki = i
    si = 1 + sum(levels) / ki
    levels.append(si)

### Analyze paths from the unique input node to the unique output node
The distribution of numbers of all paths of different lengths has significant effect on performance.
The effective path, which has been defined as ...
A larger distribution interval of path lengths